In [33]:
#basic
import pandas as pd
import numpy as np
from pandas import DataFrame
import statsmodels.api as sm

In [34]:
def dataframeCleaner(df):
    for columnname in df: #looping through titles of the table 
        temp = []     
        for column in df[columnname]:   #geting column elements for the each title
            column = str(column)
            column = column.replace(',','')# Removing unwanted data clutter
            column = column.replace('+','')#Removing unwanted '+'sign  
            try:   #using try except block to convert datatype string to integer while avoiding error
                column = int(column)
            except:
                pass
            
            temp.append(column)
        df[columnname] = temp
        
    df.replace('nan', 0, regex=True,inplace=True) # delete N/A
    df.replace(['\n'], '', regex=True, inplace=True) #delete unwanted newline
    df.replace([','], '', regex=True, inplace=True) #delete comma
    df.replace(r'^\s*$', 0, regex=True,inplace=True)# converting empty string to 0

    return df

In [35]:
def cleanerForNewCases(dataset):
    # remove unnecessary column
    df = dataset.drop(dataset.head(8).index)
    if set(['Active Cases/1M pop','New Deaths/1M pop','1 Testevery X ppl','1 Deathevery X ppl','1 Caseevery X ppl','Tests/\n1M pop','TotalTests','Deaths/1M pop','Tot Cases/1M pop','New Cases/1M pop']).issubset(df.columns):
        df.drop(['Active Cases/1M pop','New Deaths/1M pop','1 Testevery X ppl','1 Deathevery X ppl','1 Caseevery X ppl','Tests/\n1M pop','TotalTests','Deaths/1M pop','Tot Cases/1M pop','New Cases/1M pop'], inplace=True, axis=1)
    if set(['NewRecovered','NewCases','NewDeaths']).issubset(df.columns):
        df.dropna(subset=['NewRecovered'],how='any', inplace=True)
        df.dropna(subset=['NewCases'],how='any', inplace=True)
        df.dropna(subset=['NewDeaths'],how='any', inplace=True)
    # basic cleanner
    df = dataframeCleaner(df) 

    return df

In [37]:
raw_data = pd.read_csv('23_04_2021.csv')
df = cleanerForNewCases(raw_data) 
#df=df[['NewCases','Continent']]
#df.head()

## Ordinary Least Squares Regression

In [38]:
X = df['NewCases'] # one variable for simple linear regression
Y = df['NewRecovered']

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:           NewRecovered   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     3545.
Date:                Sat, 22 May 2021   Prob (F-statistic):           5.75e-85
Time:                        18:57:20   Log-Likelihood:                -1075.9
No. Observations:                 111   AIC:                             2156.
Df Residuals:                     109   BIC:                             2161.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1331.0324    385.505      3.453      0.0

In [39]:
# print the exact p-value 
for attributeIndex in range (0, 2):
    print(model.pvalues[attributeIndex])

0.0007911547049567241
5.753990650828544e-85


Từ kết quả trên ta có thể đưa ra kết luận:

- Biến NewCases có ý nghĩa đối với mô hình về mặt thống kê (p-value = 5.75e-85).

- Mô hình phù hợp tốt với dữ liệu quan sát về mặt thống kê (p-value = 5.75e-85).

- Biến NewCases có thể giải thích được 97% sự thay đổi của biến NewRecovered

- Phương trình hồi quy:

        NewRecovered = 1331.0324 + 0.970 * NewCases

## Linear regression

In [40]:
x = df['NewDeaths'] 
y = df['NewCases']

In [41]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [42]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [43]:
lr.fit(np.array(x_train).reshape(-1,1),np.array(y_train).reshape(-1,1))

LinearRegression()

In [44]:
y_pred=lr.predict(np.array(x_test).reshape(-1,1))

In [45]:
from sklearn.metrics import mean_squared_error
mean_squared_error(x_test,y_pred) 

96321112.20804888

Giá trị MSE khá lớn cho nên mô hình dự đoán này không được đánh giá tốt.